In [37]:
import torch
from kan import *
from my_functions import calculate_signature, calculate_signature_over_all_surface, format_signature
import plotly.graph_objects as go


In [38]:
mesh_path = "data/bunny_curvs.ply"
sampled_indices = random.sample(range(0, 106712), 5)
print(sampled_indices)


[50494, 99346, 55125, 5306, 33936]


In [39]:

training_num_per_sampled_surface = 5
signatures_list = []
means = []
stds = []

grid_range = 1.0
ball_radius = 0.09
lr = 1.0
dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=5306, show_sample=True, ball_radius=ball_radius, seed=5306, down_sampling_ratio=0.8)
show_sample = False
for i,sampled_index in enumerate(sampled_indices):

    signatures = torch.zeros((training_num_per_sampled_surface, 6))

    for j in range(training_num_per_sampled_surface):
        if j ==0:
            show_sample= True
        else:
            show_sample = False
        dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=sampled_index, show_sample=show_sample, ball_radius=ball_radius, seed=j, down_sampling_ratio=0.8)
        model = KAN(width=[2,5,1], grid=1, k=3, grid_eps=1.0, seed=0, grid_range=[-grid_range,grid_range], noise_scale=0.1, noise_scale_base=0.1, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
        model.train(dataset, opt="LBFGS", steps=20,lr=lr, lamb=100.0, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
        model = model.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0,1],[0]])
        model.train(dataset, opt="LBFGS", steps=50,lr=lr, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
         # manual mode
        # fix all
        model.fix_symbolic(0,0,0,'x^4')
        model.fix_symbolic(0,1,0,'x^3')
        model.fix_symbolic(1,0,0,'x^2')

        model.train(dataset, opt="LBFGS", steps=50, lr=lr,sglr_avoid=True, lamb_coef=1.0, lamb_coefdiff=1.0, loss_fn=gaussian_weighted_mse)

        print(model.symbolic_formula()[0][0])
        x = torch.tensor([[0.0], [0.0]], requires_grad=True).T

        signature = calculate_signature(model, x) # only calc sig in the origin
        print("sampled_index", i, "exp ", j, "signature ", signature)
        signatures[j,:] = signature
    signatures_list.append(signatures)



Number of vertices: 106712


Number of vertices: 106712


train loss: 1.91e-02 | test loss: 1.87e-02 | reg: 3.03e+00 : 100%|██| 20/20 [00:02<00:00,  8.68it/s]
train loss: 8.50e-03 | test loss: 7.11e-03 | reg: 3.72e+00 : 100%|██| 50/50 [00:01<00:00, 27.19it/s]


r2 is 0.8224742412567139
r2 is not very high, please double check if you are choosing the correct symbolic function.
r2 is 0.9495314955711365
r2 is 0.8988100290298462
r2 is not very high, please double check if you are choosing the correct symbolic function.


train loss: 8.75e-03 | test loss: 7.29e-03 | reg: 5.46e+00 : 100%|██| 50/50 [00:00<00:00, 69.77it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



...

r2 is 0.8373980522155762
r2 is not very high, please double check if you are choosing the correct symbolic function.
r2 is 0.9113047122955322
r2 is 0.9380620121955872


train loss: 8.44e-03 | test loss: 7.93e-03 | reg: 3.68e+00 : 100%|██| 50/50 [00:00<00:00, 66.41it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



-8691.08*((0.03 - x_2)**3 - 0.94*(-x_1 - 0.09)**4)**2
sampled_index 0 exp  1 signature  tensor([[ 0.0316, -0.1745,  6.8921, -3.8490,  2.8311, 14.2129]],
       grad_fn=<StackBackward0>)
Number of vertices: 106712


train loss: 1.70e-02 | test loss: 1.67e-02 | reg: 3.29e+00 : 100%|██| 20/20 [00:01<00:00, 11.05it/s]
train loss: 8.87e-03 | test loss: 7.07e-03 | reg: 1.98e+00 :   6%|▏  | 3/50 [00:00<00:03, 14.53it/s]


KeyboardInterrupt: 

In [ ]:
signatures

In [ ]:
#  #discard all nan and 0
signatures_list_without_nan_and_zeros = []
for signatures in signatures_list:
    signatures_without_nan = signatures[~torch.isnan(signatures).any(dim=1)]
    signatures_without_nan_zeros = signatures_without_nan[~(signatures_without_nan == 0).all(dim=1)]
    signatures_list_without_nan_and_zeros.append(signatures_without_nan_zeros)



In [ ]:
signatures_mean_per_sampled_surface = [torch.mean(signatures, dim=0) for signatures in signatures_list_without_nan_and_zeros]
signatures_std_per_sampled_surface = [torch.std(signatures, dim=0) for signatures in signatures_list_without_nan_and_zeros]


print("mean signature ", signatures_mean_per_sampled_surface)
print("std signature ", signatures_std_per_sampled_surface)

In [ ]:
signatures_list[3]

In [ ]:
# std/mean per patch

std_over_mean_per_signature = [abs(signatures_std_per_sampled_surface[i]/signatures_mean_per_sampled_surface[i]) for i in range(len(signatures_mean_per_sampled_surface))]
std_over_mean_per_signature

In [ ]:
for i in range(len(signatures_list_without_nan_and_zeros)):
    outliers = torch.abs(signatures_list_without_nan_and_zeros[i] - signatures_mean_per_sampled_surface[i]) >  1.5*signatures_std_per_sampled_surface[i]
    signatures_without_outliers = signatures_list_without_nan_and_zeros[i][~outliers.any(dim=1)]
    mean_signature = torch.mean(signatures_without_outliers, dim=0)
    std_signature = torch.std(signatures_without_outliers, dim=0)
    std_over_mean = abs(std_signature/mean_signature)
    print(signatures_without_outliers.shape)
    print(signatures_without_outliers)
    print("std over mean without outliers ", std_over_mean)






In [ ]:
outliers


In [ ]:
# calculate std/mean to see the variation of the signature
std_over_mean = abs(std_signature/mean_signature)
print("std/mean ", std_over_mean)

In [ ]:
signatures.shape

In [ ]:
mean_signature.shape

In [ ]:
a = torch.mean(signatures, dim=1)

In [ ]:
signatures - a

In [ ]:
torch.std(signatures, dim=1).shape

In [ ]:
signatures[:,~outliers.any(dim=1)].shape


In [ ]:

outliers.any(dim=1).shape